# Dragon Transport

Given your great Optimization Wizard training, the Ministry of Magic has asked that you transport 20 dragons from Romania to Hogwarts for the Triwizard tournament.   The dragons will be transported on a route through five cities, with a choice of three different modes of transport between each of the pairs of cities on the route.
The route to be followed is exactly:

1. Romania
2. Transylvania
3. Egypt
4. Godric's Hollow
5. Hogwarts

On each leg of the route, the dragons are to all be transported by Hogwarts Express (Train), Portkey, or Thestral.  In any of the intermediate cities, it is possible to change the mode of transport, but you must use a single mode of transport for all the dragons between two consecutive cities.  The following table lists the cost of transport in galleons per dragon between the pairs of cities:

|Pairs of cities|1-2 |2-3 |3-4 |4-5 |
|---------------|----|----|----|----|
|Train          |30  |25  |40  |60  |
|Portkey        |25  |40  |45  |50  |
|Thestral       |40  |20  |50  |45  |

The next table shows the cost of changing the mode of transport in galleons/dragon.  (This cost is independent of the location at which the change is made):

|From/To |Train |Portkey |Thestral |
|--------|------|--------|---------|
|Train   |0     |5       |12       |
|Portkey |8     |0       |10       |
|Thestral|15    |10      |0        |

### How should the transport be organized to minimize the cost?  What is the minimum cost for transporting the 20 dragons?

In [1]:
# Load the gams extension
%load_ext gams_magic

In [2]:
%%gams
set Modes /Train, Portkey, Thestral/; 
alias(Modes,m,mm);

set Legs /1-2, 2-3, 3-4, 4-5/;
alias(Legs,l,ll);

table clegs(Modes,Legs)
            1-2 2-3 3-4 4-5
Train       30  25  40  60 
Portkey     25  40  45  50
Thestral    40  20  50  45;

table cmodes(Modes,Modes)
            Train Portkey Thestral
Train         0     5       12 
Portkey       8     0       10  
Thestral     15    10        0;


binary variable x(m,l);
free variable y(m,l);
binary variable z(m,mm,l);
free variable TotalCost;

equations obj, xeq, yeq1, yeq2;
xeq(l).. sum(m,x(m,l)) =E= 1;
yeq1(m,mm,l)$(card(l)>1).. x(m,l) + x(mm,l+1) =L= (1-z(m,mm,l+1)) + z(m,mm,l+1)*2 ;
yeq2(m,mm,l)$(card(l)>1)..  2*z(m,mm,l+1) -2*(1-z(m,mm,l+1)) =L= x(m,l) + x(mm,l+1) ;

obj.. TotalCost =E= sum((m,l), clegs(m,l)*x(m,l)*20)
                    + sum((m,mm,l), z(m,mm,l)*cmodes(m,mm)*20 );
                    
model dragon /all/;
solve dragon using MIP min TotalCost;
display x.L, z.L, TotalCost.L;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Integer (8),3000.0,77,46,MIP,CPLEX,0.016


In [5]:
%gams_pull -d x
x.drop(['marginal','lower','upper','scale'],axis=1,inplace=True)
x.set_index('Legs',inplace=True)
display(x.loc[x['level']>0])

,Modes,level
Legs,,
1-2,Train,1.0
2-3,Train,1.0
3-4,Train,1.0
4-5,Portkey,1.0
